# QU CBE Python Workshop - Applications

**Mohammed Ait Lahcen, Department of Finance and Economics, College of Business and Economics, Qatar University**

## Pricing a European call option

In this notebook we will price a European call option under the risk-neutral valuation. We will use 3 different methods and then compare the results. 

The price of the call option is just the present discounted value of the expected payoff:

$$
C = e^{-rT} \mathbb E \max\{ S_T - K, 0 \}
$$

where

$ r $ is the risk free rate,  
$ T $ is the expiry date,  
$ K $ is the strike price and  
$ \{S_t\} $ is the price of the underlying asset at time $ t $.

In [3]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

We start by defining the initial stock price as well as other parameters:

In [4]:
S_0 = 100  # initial stock price
K = 105  # strike price
T = 1  # time-to-maturity
r = 0.05  # riskless short rate
σ = 0.2  # volatility

### Monte Carlo simulation

We use monte carlo simulation to obtain the expected value of the asset price at maturity
$$
S_T = S_0 \exp{\left((r - \frac{1}{2}\sigma^2)T + \sigma \sqrt{T} z\right)}
$$
given the normally distributed return $z$.

In [5]:
def monte_carlo_simulation(n, S_0, K, T, r, σ):

    np.random.seed(123)  # fix random numbers
    
    z = np.random.standard_normal(n)  # draw n random numbers from the return distribution (standard normal)

    S_T = S_0 * np.exp((r - 0.5 * σ ** 2) * T + σ * np.sqrt(T) * z)  # simulate n stock prices at maturity

    payoff = np.maximum(S_T - K, 0)

    call_price = np.exp(-r * T) * np.mean(payoff)  # calculate Monte Carlo estimator of the present discounted value of the expected payoff
    
    return call_price


In [6]:
n = 1000000 # number of simulations

call_price_sim = monte_carlo_simulation(n, S_0, K, T, r, σ)

call_price_sim

8.033020725360096

### Quadrature

Instead of using Monte Carlo simulation, we will use Gaussian quadrature to compute the expectation of the asset price at maturity:

In [7]:
def integrand(z, S_0, K, T, r, σ):
    
    # Map  return z to S_T for payoff calculation
    S_T = S_0 * np.exp((r - 0.5 * σ ** 2) * T + σ * np.sqrt(T) * z)
    
    # Calculate the payoff
    payoff = np.maximum(S_T - K, 0)

    return np.exp(-r * T) * payoff * norm.pdf(z)

In [8]:
call_price_quad, error = quad(integrand, -5, 5, args=(S_0, K, T, r, σ))

call_price_quad

8.021301532186943

### Black-Scholes formula

Finally, we will use the standard Black-Scholes model which gives us a closed-form solution to compute the price of the call option:
$$
C = S_0 N(d_1) - K e^{-r T} N(d_2)
$$
where $N$ is the standard normal CDF and $d_1$ and $d_2$ are given by
$$
d_1 = \frac{\ln(\frac{S_0}{K}) + (r + \frac{\sigma^2}{2})T}{\sigma\sqrt{T}}
$$
$$
d_2 = d_1 - \sigma\sqrt{T}
$$

In [9]:
def black_scholes(S_0, K, T, r, σ):

    d1 = (np.log(S_0 / K) + (r + 0.5 * σ ** 2) * T) / (σ * np.sqrt(T))
    
    d2 = d1 - σ * np.sqrt(T)
    
    call_price = S_0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    
    return call_price

call_price_bs = black_scholes(S_0, K, T, r, σ)

call_price_bs

8.021352235143176

In [10]:
# print the results
print(f'Monte Carlo call price: {call_price_sim:.4f}')
print(f'Quadrature call price: {call_price_quad:.4f}')
print(f'Black-Scholes call price: {call_price_bs:.4f}')

Monte Carlo call price: 8.0330
Quadrature call price: 8.0213
Black-Scholes call price: 8.0214
